In [1]:
# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes.
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.
import nest_asyncio

nest_asyncio.apply()

In [2]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex, SQLDatabase
from llama_index.readers.wikipedia import WikipediaReader

In [3]:
# Setting local llm 
#Use Local embeddings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.core.embeddings import resolve_embed_model
from llama_index.llms.ollama import Ollama
Settings.llm = Ollama(model="mistral", request_timeout=30.0)
Settings.embed_model = resolve_embed_model("local:BAAI/bge-small-en-v1.5")

In [4]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column,
)

In [5]:
engine = create_engine("sqlite:///:memory:", future=True)
metadata_obj = MetaData()

In [6]:
# create city SQL table
table_name = "city_stats"
city_stats_table = Table(
    table_name,
    metadata_obj,
    Column("city_name", String(16), primary_key=True),
    Column("population", Integer),
    Column("country", String(16), nullable=False),
)

metadata_obj.create_all(engine)

In [7]:
# print tables
metadata_obj.tables.keys()

dict_keys(['city_stats'])

In [8]:
from sqlalchemy import insert

rows = [
    {"city_name": "Toronto", "population": 2930000, "country": "Canada"},
    {"city_name": "Tokyo", "population": 13960000, "country": "Japan"},
    {"city_name": "Berlin", "population": 3645000, "country": "Germany"},
]
for row in rows:
    stmt = insert(city_stats_table).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

In [9]:
with engine.connect() as connection:
    cursor = connection.exec_driver_sql("SELECT * FROM city_stats")
    print(cursor.fetchall())

[('Toronto', 2930000, 'Canada'), ('Tokyo', 13960000, 'Japan'), ('Berlin', 3645000, 'Germany')]


In [10]:
cities = ["Toronto", "Berlin", "Tokyo"]
wiki_docs = WikipediaReader().load_data(pages=cities)

In [11]:
sql_database = SQLDatabase(engine, include_tables=["city_stats"])

In [12]:
from llama_index.core.query_engine import NLSQLTableQueryEngine

In [13]:
sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["city_stats"],
#   llm="None",
)

In [14]:
# build a separate vector index per city
# You could also choose to define a single vector index across all docs, and annotate each chunk by metadata
vector_indices = []
for wiki_doc in wiki_docs:
    vector_index = VectorStoreIndex.from_documents([wiki_doc])
    vector_indices.append(vector_index)

In [15]:
vector_query_engines = [index.as_query_engine() for index in vector_indices]

In [16]:
from llama_index.core.tools import QueryEngineTool


sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    description=(
        "Useful for translating a natural language query into a SQL query over"
        " a table containing: city_stats, containing the population/country of"
        " each city"
    ),
)
vector_tools = []
for city, query_engine in zip(cities, vector_query_engines):
    vector_tool = QueryEngineTool.from_defaults(
        query_engine=query_engine,
        description=f"Useful for answering semantic questions about {city}",
    )
    vector_tools.append(vector_tool)

In [17]:
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=([sql_tool] + vector_tools),
 #   llm="None",
)

In [18]:
response = query_engine.query("Which city has the highest population?")
print(str(response))

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:llama_index.core.query_engine.router_query_engine:Selecting query engine 0: Choice 1 is relevant as it allows translating a natural language query ('Which city has the highest population?') into an SQL query that can be executed over the table 'city_stats' to retrieve the answer..
Selecting query engine 0: Choice 1 is relevant as it allows translating a natural language query ('Which city has the highest population?') into an SQL query that can be executed over the table 'city_stats' to retrieve the answer..
INFO:llama_index.core.indices.struct_store.sql_retriever:> Table desc str: Table 'city_stats' has columns: city_name (VARCHAR(16)), population (INTEGER), country (VARCHAR(16)), and foreign keys: .
> Table desc str: Table 'city_stats' has columns: city_name (VARCHAR(16)), population (INTEGER), country (VARCHAR(16)), and foreign key

In [19]:
response = query_engine.query("Tell me about the historical museums in Berlin")
print(str(response))

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:llama_index.core.query_engine.router_query_engine:Selecting query engine 2: While all choices mention answering semantic questions about cities, choice 3 specifically relates to Berlin..
Selecting query engine 2: While all choices mention answering semantic questions about cities, choice 3 specifically relates to Berlin..
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:llama_index.core.query_engine.router_query_engine:Selecting query engine 1: But 'Tell me about the historical museums in Berlin' is a more specific question than just 'Answering semantic questions about Toronto', which is mentioned in choice 2..
Selecting query engine 1: But 'Tell me about the historical museums in Berlin' is a more specific question than just 'Answ

In [21]:
response = query_engine.query("Which countries are each city from?")
print(str(response))

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:llama_index.core.query_engine.router_query_engine:Selecting query engine 0: Choice 1 is relevant as it mentions a table containing city statistics with population and country. This suggests that this choice would allow us to retrieve the country information for each city..
Selecting query engine 0: Choice 1 is relevant as it mentions a table containing city statistics with population and country. This suggests that this choice would allow us to retrieve the country information for each city..
INFO:llama_index.core.indices.struct_store.sql_retriever:> Table desc str: Table 'city_stats' has columns: city_name (VARCHAR(16)), population (INTEGER), country (VARCHAR(16)), and foreign keys: .
> Table desc str: Table 'city_stats' has columns: city_name (VARCHAR(16)), population (INTEGER), country (VARCHAR(16)), and foreign keys: .
INFO:httpx: